In [1]:
import numpy as np
import robosuite as suite
from robosuite.environments.manipulation.empty import Empty
from scipy import interpolate
from robosuite.utils.mjmod import DynamicsModder

In [2]:
horizon = 200

In [3]:
def plan(start_pose, middle_pose, end_pose, horizon):
    cs = interpolate.CubicSpline([0, horizon // 2, horizon], [start_pose, middle_pose, end_pose], 
                                       axis=0, bc_type='clamped')
    return cs(range(horizon))  
def difference(traj):
    traj = np.array(traj)
    new_traj = [traj[0]]
    for i in range(len(traj)):
        if i == 0:
            continue
        new_traj.append(traj[i] - traj[i - 1])
    return new_traj

In [4]:
suite.environments.base.register_env(Empty)

In [5]:
controller_config = suite.load_controller_config(default_controller="JOINT_POSITION")
controller_config["damping_ratio"] = 0.1
controller_config["ramp_ratio"] = 0.1
print(controller_config)

{'type': 'JOINT_POSITION', 'input_max': 1, 'input_min': -1, 'output_max': 0.05, 'output_min': -0.05, 'kp': 50, 'damping_ratio': 0.1, 'impedance_mode': 'fixed', 'kp_limits': [0, 300], 'damping_ratio_limits': [0, 10], 'qpos_limits': None, 'interpolation': None, 'ramp_ratio': 0.1}


In [6]:
env = suite.make(
    "Empty",
    robots="IIWA",             # load a Sawyer robot and a Panda robot
    gripper_types="ClothGripper",                # use default grippers per robot arm
    controller_configs=controller_config, 
    has_renderer=True,                      # on-screen rendering
    render_camera="sideview",              # visualize the "frontview" camera
    has_offscreen_renderer=False,           # no off-screen rendering
    render_collision_mesh=True,
    control_freq=20,                        # 20 hz control for applied actions
    horizon=horizon,                            # each episode terminates after 200 steps
    use_object_obs=False,                   # no observations needed
    use_camera_obs=False,                   # no observations needed
)

Creating window glfw


In [7]:
import tqdm
import random

num_dataset_size = 1000
num_cloth_joints = 11

all_geom_positions = []
all_parameters = []
for _ in tqdm.tqdm(range(num_dataset_size)):
    done = False
    obs = env.reset()
    
    modder = DynamicsModder(sim=env.sim)
    damping = random.random() * 20
    stiffness = random.random() * 20
    all_parameters.append([damping, stiffness])
    
    for i in range(num_cloth_joints):
        modder.mod("gripper0_joint" + str(i), "damping", 3)
        modder.mod("gripper0_joint" + str(i), "stiffness", 0)
    
    geom_positions = []
    traj = difference(plan([0, 0, 0, 0, 0, 0, 0], [0, -10, 0, 10, 0, -10, 0], [0, 0, 0, 0, 0, 0, 0], horizon))
    
    initial_jpos = np.array([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) for joint in range(1, 8)])
    final_jpos = np.array([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) + (0.4 if joint == 2 else 0) for joint in range(1, 8)])
    
    while not done:
        action = traj[i]
        jpos = np.array([env.sim.data.get_joint_qpos("robot0_joint_" + str(joint)) for joint in range(1, 8)])
        relative_jpos = final_jpos - jpos
        
        for j in range(num_cloth_joints):
            pos = env.sim.data.geom_xpos[env.sim.model.geom_name2id("gripper0_g{}_col".format(j))]
            geom_positions.append(pos[0])
            geom_positions.append(pos[2])
            
        obs, reward, done, info = env.step(relative_jpos) 
        env.render()
    all_geom_positions.append(geom_positions)
    
all_geom_positions = np.array(all_geom_positions)
all_parameters = np.array(all_parameters)

  0%|          | 0/1000 [00:00<?, ?it/s]

Creating window glfw
[0.  0.4 0.  0.  0.  0.  0. ]
[ 4.22156352e-08  3.98741441e-01  1.85883932e-07  9.23438577e-05
  1.80471756e-07 -1.12452197e-04  4.73409564e-09]
[ 7.73250573e-08  3.95263295e-01  3.00613658e-07  2.60917503e-04
  2.58977074e-07 -2.46674772e-04  6.43504086e-09]
[ 1.03568159e-07  3.89869922e-01  3.35824928e-07  4.20636249e-04
  2.11135381e-07 -2.73397422e-04  9.40147993e-09]
[ 1.26776440e-07  3.82834058e-01  3.30615876e-07  5.25756797e-04
  8.98555928e-08 -2.74729354e-04  1.34136406e-08]
[ 1.47979615e-07  3.74402135e-01  3.09992495e-07  5.66094184e-04
 -4.97697673e-08 -2.75458339e-04  1.79273695e-08]
[ 1.60207126e-07  3.64801180e-01  2.78113017e-07  5.70303655e-04
 -1.42034649e-07 -2.75919775e-04  2.20816081e-08]
[ 1.70317603e-07  3.54237383e-01  2.47874548e-07  5.72755589e-04
 -2.11262610e-07 -2.76487712e-04  2.62393506e-08]
[ 1.83854130e-07  3.42890441e-01  2.28702921e-07  5.77273664e-04
 -2.78033647e-07 -2.77369719e-04  3.06798935e-08]
[ 2.01145290e-07  3.30921596e

[ 6.56350591e-07  3.71917816e-03 -3.27339172e-06  7.66056503e-04
 -3.65528935e-06 -2.89517686e-04  3.34894984e-07]
[ 6.58693847e-07  3.71872769e-03 -3.29379508e-06  7.66326884e-04
 -3.66301375e-06 -2.89650252e-04  3.36121474e-07]
[ 6.61188968e-07  3.71827425e-03 -3.31382262e-06  7.66609530e-04
 -3.67086993e-06 -2.89795421e-04  3.37368841e-07]
[ 6.63821658e-07  3.71781816e-03 -3.33351753e-06  7.66902949e-04
 -3.67885942e-06 -2.89951633e-04  3.38636193e-07]
[ 6.66574230e-07  3.71735982e-03 -3.35293185e-06  7.67205346e-04
 -3.68698201e-06 -2.90117015e-04  3.39922339e-07]
[ 6.69427955e-07  3.71689965e-03 -3.37211989e-06  7.67514854e-04
 -3.69523616e-06 -2.90289619e-04  3.41225950e-07]
[ 6.72364672e-07  3.71643804e-03 -3.39113368e-06  7.67829686e-04
 -3.70361939e-06 -2.90467581e-04  3.42545685e-07]
[ 6.75367812e-07  3.71597535e-03 -3.41002020e-06  7.68148230e-04
 -3.71212860e-06 -2.90649224e-04  3.43880268e-07]
[ 6.78422960e-07  3.71551190e-03 -3.42881984e-06  7.68469102e-04
 -3.72076036e-0

[ 9.06050810e-07  3.68455695e-03 -4.70531161e-06  7.88141361e-04
 -4.46652231e-06 -3.01108915e-04  4.51233084e-07]
[ 9.09624912e-07  3.68409734e-03 -4.72450815e-06  7.88413635e-04
 -4.47914875e-06 -3.01240452e-04  4.52948138e-07]
[ 9.13201602e-07  3.68363781e-03 -4.74370530e-06  7.88685445e-04
 -4.49180006e-06 -3.01371558e-04  4.54665196e-07]
[ 9.16780808e-07  3.68317835e-03 -4.76290295e-06  7.88956797e-04
 -4.50447572e-06 -3.01502239e-04  4.56384207e-07]
[ 9.20362459e-07  3.68271897e-03 -4.78210101e-06  7.89227695e-04
 -4.51717520e-06 -3.01632499e-04  4.58105119e-07]
[ 9.23946486e-07  3.68225966e-03 -4.80129938e-06  7.89498142e-04
 -4.52989800e-06 -3.01762344e-04  4.59827879e-07]
[ 9.27532820e-07  3.68180043e-03 -4.82049796e-06  7.89768144e-04
 -4.54264360e-06 -3.01891779e-04  4.61552440e-07]
[ 9.31121395e-07  3.68134128e-03 -4.83969667e-06  7.90037704e-04
 -4.55541154e-06 -3.02020810e-04  4.63278752e-07]
[ 9.34712148e-07  3.68088220e-03 -4.85889542e-06  7.90306827e-04
 -4.56820132e-0

  0%|          | 1/1000 [00:04<1:08:23,  4.11s/it]

...

[ 4.90298792e-07  3.76259844e-03 -2.38883090e-06  8.13311041e-04
 -2.99659226e-06 -3.33163925e-04 -4.96180081e-08]
[ 4.92073274e-07  3.76214827e-03 -2.40679167e-06  8.13556867e-04
 -3.00218116e-06 -3.33274762e-04 -5.11590505e-08]
[ 4.93849149e-07  3.76169493e-03 -2.42491070e-06  8.13815882e-04
 -3.00797140e-06 -3.33399150e-04 -5.26848215e-08]
[ 4.95628101e-07  3.76123859e-03 -2.44317623e-06  8.14087227e-04
 -3.01396116e-06 -3.33536191e-04 -5.41954073e-08]
[ 4.97412226e-07  3.76077958e-03 -2.46156841e-06  8.14369414e-04
 -3.02014269e-06 -3.33684334e-04 -5.56912528e-08]
[ 4.99203729e-07  3.76031831e-03 -2.48006343e-06  8.14660648e-04
 -3.02650466e-06 -3.33841707e-04 -5.71730220e-08]
[ 5.01004706e-07  3.75985519e-03 -2.49863657e-06  8.14959055e-04
 -3.03303407e-06 -3.34006360e-04 -5.86414906e-08]
[ 5.02817018e-07  3.75939062e-03 -2.51726425e-06  8.15262840e-04
 -3.03971753e-06 -3.34176423e-04 -6.00974686e-08]
[ 5.04642218e-07  3.75892496e-03 -2.53592524e-06  8.15570380e-04
 -3.04654218e-0

  0%|          | 3/1000 [00:16<1:29:14,  5.37s/it]

[ 6.57966799e-07  3.72685179e-03 -3.77278719e-06  8.35077853e-04
 -3.67730209e-06 -3.44412966e-04 -1.46213051e-07]


SystemExit: 0

/home/yiliu77/Desktop/Python/envs/main_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3435: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Have some prior distribution of magazine parameters
# Model predicts parameters distribution from actions and movement
# Each cycle
# Sample parameters and choose actions to minimize entropy of predictions

In [ ]:
# Use DIAYN for trajectories

In [ ]:
all_geom_positions.shape, all_parameters.shape

In [ ]:
with open("all_geom_positions.npy", 'r') as f:
    all_geom_positions = np.load(f, all_geom_positions)
with open("all_parameters.npy", 'r') as f:
    np.load(f, all_parameters)

In [ ]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

train_split = 0.8
train_length = int(train_split * len(all_geom_positions))
val_length = len(all_geom_positions) - train_length
all_dataset = torch.utils.data.TensorDataset(torch.from_numpy(all_geom_positions), torch.from_numpy(all_parameters))
train_dataset, val_dataset = torch.utils.data.random_split(all_dataset, [train_length, val_length])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, pin_memory=True)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(4400, 2048),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(2048, 1024),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.1),
    torch.nn.Linear(1024, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 2 * 2),
).to("cuda")
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
loss_fn = torch.nn.GaussianNLLLoss()
for epoch in range(100):
    model.train()
    train_losses = []
    for position, target_params in train_loader:
        position = position.to("cuda").float()
        target_params = target_params.to("cuda").float()
        
        pred_params_mu, pred_params_logvar = torch.split(model(position), 2, dim=1)
        loss = loss_fn(pred_params_mu, target_params, torch.exp(pred_params_logvar))
        
        train_losses.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    model.eval()
    with torch.no_grad():
        val_losses = []
        for position, target_params in val_loader:
            position = position.to("cuda").float()
            target_params = target_params.to("cuda").float()

            pred_params_mu, pred_params_logvar = torch.split(model(position), 2, dim=1)
            loss = loss_fn(pred_params_mu, target_params, torch.exp(pred_params_logvar))

            val_losses.append(loss.item())
        
    print("Epoch: {}, Train Loss: {}, Val Loss: {}".format(epoch, np.mean(train_losses), np.mean(val_losses)))

In [ ]:
predictions = []
ground_truth = []
for i, (position, target_params) in enumerate(val_loader):
    prediction, _ = torch.split(model(position.to("cuda").float()), 2, dim=1)
    predictions.append(prediction.detach().cpu().numpy())
    ground_truth.append(target_params.cpu().numpy())
predictions = np.concatenate(predictions, axis=0)
ground_truth = np.concatenate(ground_truth, axis=0)

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(ground_truth[:, 0], predictions[:, 0])
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.scatter(ground_truth[:, 1], predictions[:, 1])
plt.show()